## 1. Intro to Recursion 
A recursive sequence is defined such that there exists a function $f$ such that $a_n=f(a_{n-1},a_{n-2},...,a_{n-k})$ where $k$ is the order of the recursive sequence. This defintion sounds scary but there is actually familiar and easy to understand example of this, the fibbanaci sequence. The fibbanci sequence is defined to be $$f_{n-2}+f_{n-1}=f_n$$
This is actually just one of many different recursive sequences, and beneath we have written code for calculating the $n\text{th}$ elemenent of a linear recursive relation. 

In [ ]:
import mpmath as mp
from mpmath import mpf
mp.dps = 50 

recursive_relation = [mp.mpf(1), mp.mpf(1),mp.mpf(-1)]  
seed = [mp.mpf(1), mp.mpf(1)]  

def sliding_window(recursive_relation, seeds, n):
    state = seeds[:]
    for _ in range(n - len(seeds)):
        next_val = sum(state[i] * recursive_relation[i] for i in range(len(state)))
        state = state[1:] + [next_val]
    return state[-1]

print(sliding_window(recursive_relation, seed, 10)) 


55.0


## 2. Finding explict formulas for arbitrary linear recursive sequences
You may also be familar with an alternate formula for the fibbanaci sequence as $F_n = \frac{1}{\sqrt{5}} \left( \left(\frac{1 + \sqrt{5}}{2}\right)^n - \left(\frac{1 - \sqrt{5}}{2}\right)^n \right)$. There is actually a generalized way to find these explict formulas for linear recursive relationships.

Imagine we have some recursive relationship $a_n=k_1a_{n-1}+k_2a_{n-2}+...+k_qa_{n-q}$. Additionally image we have some generating function such that $G(X)=\sum_{n=0}^{\infty}a_nx^n$. Notice how if we rearrange our recursive relation we can see the identity that $a_n-k_1a_{n-1}-k_2a_{n-2}-...-k_qa_{n-q}=0$, lets define a polynomial $P(x)=a_nx^0-k_1a_{n-1}x^1-k_2a_{n-2}x^2-...-k_qa_{n-q}x^q$. Now if we multiply $P(x)$ and $G(x)$ we find that the result is finite, because the way that polynomial multiplication works results in all $x^n$ where $n>q$ to have a coefficent of $0$, and we can calculate the non-zero part of the this resulting polynomial which we can call $Q(x)$

Now we know that $P(x)G(x)=Q(x)$ we can rearrange this expression to tell us that $G(X)=\frac{Q(x)}{P(x)}$ we can simplify this using partial fractions but in order to do that we need to be able to computationally find the roots of $P(x)$, for for the next part of this jupiter notebook we will focus on finding roots of polynomials of any degree

# 2.1 Finding roots of high degree polynomials
It is a pretty well known fact within the mathematical community that there is no quintic equation using only addition, multiplication and exponentionals. But there are computation ways that approximate the roots of a degree five or higher polynomial. One such example is using QR decomposition to solve something known as the companion matrix which is defined to be $$C = \begin{bmatrix} 0 & 1 & 0 & \cdots & 0 \\ 0 & 0 & 1 & \cdots & 0 \\ \vdots & \vdots & \vdots & \ddots & \vdots \\0 & 0 & 0 & \cdots & 1 \\ c_0 & c_1 & c_2 & \cdots & c_{n-1}\end{bmatrix}$$ where our polynomial that we walk to solve the roots of is $c_0 + c_1x + c_2x^1+...+c_{n-1}x^{n-1}$. The eigenvalues of this matrix are actually the roots of these polynomials. 

You might be wondering now, why does this help since usally to find an eigenvalue you need to solve the charecteristic polynomial, but we can actually find eigenvalues a different way. By QR decomposition we can write an invertible matrix $A$ as $QR$ where $Q$ is orthogonal and $R$ is triangular, additionally we can see that $Q^{-1}AQ=Q^{-1}QRQ=RQ$. This proves that $Q^{-1}AQ$ and $RQ$ have the same eigenvalues, and by well known properties of determinates it also means that $A$ and $RQ$ have the same eigenvalues. 

Then by repeatedly doing this, taking our matrix $A_{n}$ and then placing it into the form of $A_{n}=QR$ and then defining $A_{n+1}=RQ$, after enough iterations we will eventually have a triangular matrix, or at least close enough for our liking. Additionally it is a well known fact that the diagonal entires of a triangular matrix are the eigenvalues(and thus roots of our polynomial) so we have sucessfully found roots to our polynomial

Beneath I have writen code to do this process, I chose not to use the numpy package because it has a build in function that does QR decomposition and I didn't think that it would be very conducive to my learning experience doing that. 

In [131]:
def dot(u, v):
    return sum(mpf(ux) * mpf(vx) for ux, vx in zip(u, v))

def scalar_mult(c, v):
    c = mpf(c)
    return [c * mpf(x) for x in v]

def vector_sub(u, v):
    return [mpf(ux) - mpf(vx) for ux, vx in zip(u, v)]

def norm(v):
    return mp.sqrt(dot(v, v))

def transpose(M):
    n = len(M)
    m = len(M[0])
    return [[M[row][col] for row in range(n)] for col in range(m)]

def mat_mult(A, B):
    p = len(A)
    q = len(A[0])
    r = len(B[0])
    result = [[mpf('0')] * r for _ in range(p)]
    for i in range(p):
        for j in range(r):
            s = mpf('0')
            for k in range(q):
                s += mpf(A[i][k]) * mpf(B[k][j])
            result[i][j] = s
    return result

def gram_schmidt(A):
    n = len(A)
    columns = [[mpf(A[row][col]) for row in range(n)] for col in range(n)]
    orthogonal = []
    R = [[mpf('0')] * n for _ in range(n)]
    for j in range(n):
        v = columns[j]
        for i in range(j):
            R[i][j] = dot(orthogonal[i], v)
            proj = scalar_mult(R[i][j], orthogonal[i])
            v = vector_sub(v, proj)
        R[j][j] = norm(v)
        q = scalar_mult(1 / R[j][j], v)
        orthogonal.append(q)
    Q = [[orthogonal[col][row] for col in range(n)] for row in range(n)]
    return Q, R

def qr_algorithm(A, max_iters=2000, tol=mpf('4e-100')):
    n = len(A)
    Ak = [[mpf(x) for x in row] for row in A]
    for _ in range(max_iters):
        Q, R = gram_schmidt(Ak)
        Ak_next = mat_mult(R, Q)

        off_diag_sum = mpf('0')
        for i in range(n):
            for j in range(n):
                if i != j:
                    off_diag_sum += mpf(abs(Ak_next[i][j]))
        if off_diag_sum < tol:
            break
        Ak = Ak_next
    eigenvalues = [Ak[i][i] for i in range(n)]
    return eigenvalues

# Example matrix
A = [
    [4.0, 1.0, -2.0],
    [1.0, 2.0, 0.0],
    [-2.0, 0.0, 3.0]
]

eigenvals = qr_algorithm(A)
print("Approximate eigenvalues:")
for val in eigenvals:
    print(val)



Approximate eigenvalues:
5.7320508075688772935274463415058723669428052538103
2.2679491924311227064725536584941276330571947461896
0.99999999999999999999999999999999999999999999999999


Now beneath I define a function which creates the companion matrix and then by finding the eigenvalues of the companion matrix we have sucessfully found the roots of this polynomial, even though they are hard to read in rational form these are the golden ratio $\phi = \frac{1+\sqrt{5}}{2}$ and the silver ratio $\delta = \frac{1-\sqrt{5}}{2}$

In [132]:
def create_companion_matrix(recursive_relation):
    n = len(recursive_relation) - 1
    comp_matrix = [[mpf('0') for _ in range(n)] for _ in range(n)]
    last_row = [mpf(x) for x in recursive_relation[:-1]]
    comp_matrix[-1] = last_row
    for i in range(n - 1):
        comp_matrix[i][i+1] = mpf('1')
    return comp_matrix

companion_matrix = create_companion_matrix(recursive_relation)
roots = qr_algorithm(companion_matrix)
print(roots)


[mpf('1.6180339887498948482045868343656381177203091798057531'), mpf('-0.61803398874989484820458683436563811772030917980576518')]


## Using the Vandermounde Matrix to find an explict formula
Now that we have our generating function written in the form of $\frac{Q(x)}{(x-r_1)(x-r_2)...(x-r_n)}$, we can do partial fraction decomposition find the explict formula. To do this we use the Vandermounde Matrix which is of the form 
$$ V = \begin{bmatrix}
1 & 1 & 1 & \cdots & 1 \\
r_1 & r_2 & r_3 & \cdots & r_k \\
r_1^2 & r_2^2 & r_3^2 & \cdots & r_k^2 \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
r_1^{k-1} & r_2^{k-1} & r_3^{k-1} & \cdots & r_k^{k-1}
\end{bmatrix}$$
When we solve for $V\begin{bmatrix}\alpha_1 \\ \alpha_2 \\ ... \\ \alpha_n\end{bmatrix} =\begin{bmatrix}a_1 \\ a_2 \\ ... \\ a_n\end{bmatrix}$ where $a_n$ is our $n^\text{th}$ initial term, we have sucessfully decomposed $\frac{Q(x)}{P(x)}$ and can now write it in the form $\sum\frac{\alpha_i}{1-r_i x}$ which can then be further rewritten into the form $a_n = \sum_i a_i(r_i)^n$

Within the code beneath, we calculate this explict formula and then compare it to the sliding window answer which we know to be accurate. We can see that calculating using the explict formula is about $10$ times faster than the sliding window approach 

In [135]:
def solve_coefficients(roots, seeds):
    k = len(roots)
    #create a vandermound Matrix 
    V = [[mpf(r)**n for r in roots] for n in range(k)]
    
    #gaussian elimation to solve for a vector 
    def gaussian_elimination(A, b):
        n = len(A)
        for i in range(n):
            max_row = max(range(i, n), key=lambda r: abs(A[r][i]))
            A[i], A[max_row] = A[max_row], A[i]
            b[i], b[max_row] = b[max_row], b[i]
            
            pivot = A[i][i]
            for j in range(i, n):
                A[i][j] = A[i][j] / pivot
            b[i] = b[i] / pivot
            
            for r in range(i+1, n):
                factor = A[r][i]
                for c in range(i, n):
                    A[r][c] = A[r][c] - factor * A[i][c]
                b[r] = b[r] - factor * b[i]
        
        x = [mpf('0')] * n
        for i in reversed(range(n)):
            s = mpf('0')
            for j in range(i+1, n):
                s += A[i][j] * x[j]
            x[i] = b[i] - s
        return x

    A = [row[:] for row in V]
    b = [mpf(val) for val in seeds]
    alpha = gaussian_elimination(A, b)

    return alpha

def explicit_formula(n, roots, coeffs):
    n = n - 1
    s = mpf('0')
    for i in range(len(roots)):
        s += coeffs[i] * (mpf(roots[i]) ** n)
    return mp.nint(s)  

def get_formula_with_symbolic_n(roots, coeffs):
    terms = []
    for alpha, r in zip(coeffs, roots):
        alpha_f = float(alpha)
        r_f = float(r)
        terms.append(f"({alpha_f:.4g})*({r_f:.4g})^n")
    formula_str = " + ".join(terms)
    return f"a_n = {formula_str}"


coeffs = solve_coefficients(roots, seed)

print(get_formula_with_symbolic_n(roots,coeffs))

import time
n = 900

start = time.perf_counter()
val = explicit_formula(n, roots, coeffs)
end = time.perf_counter()
print(f"a_{str(n)} = {float(val):.0f}")
print(f"Time elapsed: {end - start:.6f} seconds\n")

print("Output for sliding_window")
start = time.perf_counter()
val = sliding_window(recursive_relation, seed, n)
end = time.perf_counter()
print(f"a_{str(n)} = {float(val):.0f}")
print(f"Time elapsed: {end - start:.6f} seconds")


a_n = (0.7236)*(1.618)^n + (0.2764)*(-0.618)^n
a_900 = 54877108839479998090647412531102638104445589146584855692253902582276255815132534179828230161851789735991474983616239084796927383819399851902053945370082952584299216510205958645069784285184
Time elapsed: 0.000108 seconds

Output for sliding_window
a_900 = 54877108839479998090647412531102638104445589146584855692253902582276255815132534179828230161851789735991474983616239084796927383819399851902053945370082952584299216510205958645069784285184
Time elapsed: 0.004183 seconds
